In [ ]:
## THIS CELL SHOULD BE IN ALL VSCODE NOTEBOOKS ##

MARKET = 'NSE'

# Add `src` to _src.pth in .venv to allow imports in VS Code
from sysconfig import get_path
from pathlib import Path
if 'src' not in Path.cwd().parts:
    src_path = str(Path(get_path('purelib')) / '_src.pth')
    with open(src_path, 'w') as f:
        f.write(str(Path.cwd() / 'src\n'))

# Start the Jupyter loop
from ib_insync import util, IB
util.startLoop()

In [ ]:
# Set the root
from from_root import from_root
ROOT = from_root()

from utils import Vars
_vars = Vars(MARKET)
PORT = _vars.PORT
PAPER = _vars.PAPER 
OPT_COLS = _vars.OPT_COLS[0]
DATAPATH = ROOT / 'data' / MARKET.lower()

# Build `states`

In [ ]:
# Imports
import asyncio

import numpy as np
import pandas as pd
from ib_insync import MarketOrder

from utils import (get_dte, get_open_orders, get_pickle,
                   get_portfolio_with_margins, get_prices_with_ivs,
                   make_ib_contracts, move_column)

pd.options.display.max_columns = None

## Get margins of pf positions

In [ ]:
df_pfm = get_portfolio_with_margins(MARKET)

## Get open orders

In [ ]:
# Get open orders
with IB().connect(port=PORT) as ib:
    df_openords = asyncio.run(get_open_orders(ib)).set_index('conId')

## Combine positions and orders

In [ ]:
# Combine positions and orders
df_p = df_pfm.reset_index()
df_p.drop(columns=['contract', 'rePnL'], inplace=True)
df_p.rename(columns={'multiplier': 'mult'}, inplace=True)
df_p.insert(0, 'xnType', 'position')

df_o = df_openords.reset_index()
df_o.insert(0, 'xnType', 'order')
df_o.drop(columns=['orderId', 'order', 'permId'], inplace=True)
df_o.rename(columns={'totalQuantity': 'qty'}, inplace=True)

## Get underlying prices of portfolio option positions

In [ ]:
df = pd.concat([df_p, df_o], ignore_index=True)
df = df.sort_values(['symbol', 'expiry', 'xnType'], ascending=[True, True, False])

# get days to expiry
dte = df.expiry.apply(lambda x: get_dte(x, MARKET))
dte[dte < 0] = 0
df = df.assign(dte = dte)

# for orders fill position with quantities
df.position.fillna(df.qty, inplace=True)

# insert `state` column
df.insert(1, 'state', 'tbd')

# get underlying prices
undPrice = np.where(df.secType == 'OPT', np.nan, df.mktPrice)
df.insert(4, 'undPrice', undPrice)
prices_dict = df.dropna(subset='undPrice').set_index('symbol').mktPrice.to_dict()
df.undPrice = df.symbol.map(prices_dict)

# get prices for missing symbols
symbols = set(df[df.undPrice.isnull()].symbol)
und_contracts = make_ib_contracts(symbols, MARKET)
und_prices = asyncio.run(get_prices_with_ivs(port=PORT, input_contracts = und_contracts))

# merge prices dictionaries
prices_dict = prices_dict | und_prices.set_index('symbol').price.to_dict()
df.undPrice = df.symbol.map(prices_dict)

# Unreaped
### A naked call or put option that doesn't have an open order to reap. [light-yellow]

In [ ]:
# Get the reaped options
reap_mask = df.groupby('conId').position.transform(lambda x: sum(x) == 0)
df.loc[reap_mask, 'state'] = 'reaped'

# Make remaining option positions `unreaped`
unreap_state = (df.xnType == 'position') & (df.state != 'reaped') & (df.secType == 'OPT')
df.loc[unreap_state, 'state'] = 'unreaped'

In [ ]:
# Generate the unreaped db
df_unreaped = df[df.state == 'unreaped'].sort_values('mktPrice')
df_unreaped

### Check when the trades were made

In [ ]:
# check when the trades were registered
reppath = f"{MARKET.lower()}_ib_reports.pkl"
REPORTPATH = DATAPATH.parent / 'master' / reppath

reports = get_pickle(REPORTPATH)
df_report = reports['trades'].sort_values('time', ascending = False)

cond = df_report.symbol.isin(set(df_unreaped.symbol))
df_rep = df_report[cond].groupby('symbol').head(2)

df_rep.expiry = pd.to_datetime(df_rep.expiry, yearfirst=True)


### find out why `ABB` is missing in the report!

In [ ]:
df_temp = reports['positions']
df_temp[df_temp.symbol.isin(['ABB'])]

In [49]:
# merge to get order time
merge_fields = ['symbol', 'strike', 'right', 'expiry']
rep_fields = merge_fields + ['qty', 'time', 'code']

df_r = df_unreaped.merge(df_rep[rep_fields], on = merge_fields, suffixes = [None, "_ordered"])
df_r = df_r.rename(columns={'time': 'time_ordered'})
df_reap = move_column(df_r, 'qty_ordered', 13)
df_reap = move_column(df_reap, 'time_ordered', 6)
df_reap.time_ordered = pd.to_datetime(df_reap.time_ordered).dt.date

In [53]:
# determine the action and quantity
action = np.where(df_reap.position < 1, 'BUY', 'SELL')
qty = -df_reap.position
df_reap = df_reap.assign(action = action, qty = qty)

In [54]:
df_reap

,xnType,state,conId,secType,undPrice,symbol,time_ordered,expiry,strike,dte,right,mult,avgCost,mktPrice,qty_ordered,position,unPnL,margin,comm,lot_size,action,qty,lmtPrice,status,code
0,position,unreaped,669195021,OPT,1406.75,PVRINOX,2024-02-08,2024-02-29,1600.0,3.189995,C,1.0,6.034566,0.050000,-407.0,-407.0,2435.72,-135361.17,20.0,407.0,BUY,407.0,NaN,NaN,O
1,position,unreaped,669134269,OPT,152.45,FEDERALBN,2024-02-15,2024-02-29,179.0,3.189995,C,1.0,0.944120,0.050000,-5000.0,-5000.0,4470.60,-74224.68,20.0,5000.0,BUY,5000.0,NaN,NaN,O
2,position,unreaped,669153116,OPT,147.00,IEX,2024-02-06,2024-02-29,167.0,3.189995,C,1.0,2.640471,0.109116,-3750.0,-3750.0,9492.58,-98546.83,20.0,3750.0,BUY,3750.0,NaN,NaN,O
3,position,unreaped,669124164,OPT,293.30,CROMPTON,2024-02-08,2024-02-29,330.0,3.189995,C,1.0,1.385179,0.128763,-1800.0,-1800.0,2261.55,-28242.22,20.0,1800.0,BUY,1800.0,NaN,NaN,O
4,position,unreaped,669123912,OPT,293.30,CROMPTON,2024-02-08,2024-02-29,250.0,3.189995,P,1.0,0.686034,0.154016,-1800.0,-1800.0,957.63,-22842.22,20.0,1800.0,BUY,1800.0,NaN,NaN,O
5,position,unreaped,669215336,OPT,1714.90,UBL,2024-02-08,2024-02-29,1500.0,3.189995,P,1.0,3.037214,0.329203,-400.0,-400.0,1083.20,-77190.10,20.0,400.0,BUY,400.0,NaN,NaN,O
6,position,unreaped,669205757,OPT,976.50,TATACHEM,2024-02-08,2024-02-29,1090.0,3.189995,C,1.0,5.050863,0.365640,-550.0,-550.0,2576.87,-113233.57,20.0,550.0,BUY,550.0,NaN,NaN,O
7,position,unreaped,669198678,OPT,739.05,SBICARD,2024-02-08,2024-02-29,800.0,3.189995,C,1.0,3.066714,0.427587,-800.0,-800.0,2111.30,-103467.95,20.0,800.0,BUY,800.0,NaN,NaN,O
8,position,unreaped,669151044,OPT,17.55,IDEA,2024-02-07,2024-02-29,18.0,3.189995,C,1.0,0.249400,0.435625,-80000.0,-80000.0,-14898.00,-544667.49,20.0,80000.0,BUY,80000.0,NaN,NaN,O
9,position,unreaped,669153681,OPT,437.80,IGL,2024-02-08,2024-02-29,390.0,3.189995,P,1.0,1.381127,0.479930,-1375.0,-1375.0,1239.15,-81001.13,20.0,1375.0,BUY,1375.0,NaN,NaN,O


# Uncovered
### A (long/short) stock with no covered (call/put) buy orders

In [ ]:
df[['symbol', 'position', 'mult']]

In [ ]:
# Get the covered options
df.groupby('symbol')[['position', 'mult']].prod()

# Unsowed

In [ ]:
# get unds, open orders and portfolio
unds = set(get_pickle(DATAPATH / 'unds.pkl').keys())
df_openorder, df_pf = asyncio.run(get_order_pf(PORT))

In [ ]:
# No orders to sow and no existing positions

options = df_pf.secType == 'OPT'
stocks = df_pf.secType == 'STK'

long = df_pf.position > 0
long_options = long & options
df_pf[long_options]

unsowed = unds - set(df_openorder.symbol) - set(df_pf.symbol)
unsowed

# Orphaned

In [ ]:
# Long calls or puts without any underlying stock position
options = df.secType == 'OPT'
stocks = df.secType == 'STK'
long = df.position >= 1
long_stocks = long & stocks
long_options = long & options
orphaned = long_stocks & long_options

orphaned = df[orphaned]
orphaned